<a href="https://colab.research.google.com/github/Malleshcr7/AI-ML-Projects/blob/main/Face_recognition_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-Time Face Recognition System for Google Colab


## Install required packages
Run this cell once at the start to install libraries needed for face recognition and image handling.


In [ ]:
# Install required packages
!pip install opencv-python cmake dlib face_recognition

import cv2
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import os
import pickle
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import time

print("All packages installed successfully!")

## Initialize Global Variables



In [13]:
# Initialize global variables at the start
known_face_encodings = []
known_face_names = []

print("Global variables initialized!")

Global variables initialized!


#Simple Face Detection Function

In [ ]:
def detect_faces_simple(image_path):
    """Simple face detection using Haar Cascade"""
    try:
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not load image {image_path}")
            return None

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Load face cascade
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw rectangles around faces
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, "Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        print(f"Detected {len(faces)} face(s)")
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    except Exception as e:
        print(f"Error in face detection: {e}")
        return None

# Test the function
print("Testing face detection...")
print("Upload an image for face detection:")
uploaded_files = files.upload()

for filename in uploaded_files.keys():
    result = detect_faces_simple(filename)
    if result is not None:
        plt.figure(figsize=(10, 8))
        plt.imshow(result)
        plt.title(f"Face Detection: {filename}")
        plt.axis('off')
        plt.show()

#Webcam Photo Capture

In [13]:
def take_photo(filename='photo.jpg', quality=0.8):
    """Take a photo using webcam"""
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))
    # Decode base64 image
    binary = b64decode(data.split(',')[1])

    # Save image
    with open(filename, 'wb') as f:
        f.write(binary)

    return filename

def capture_and_detect_faces():
    """Capture photos and detect faces"""
    print("=== WEBCAM FACE DETECTION ===")
    print("You will be prompted to take 3 photos.")
    print("Click the 'Capture' button when ready for each photo.")

    for i in range(3):
        print(f"\n Ready for photo {i+1}/3...")
        filename = f'webcam_photo_{i}.jpg'

        try:
            # Take photo
            photo_path = take_photo(filename)
            print(" Photo captured successfully!")

            # Detect faces in the photo
            result_image = detect_faces_simple(photo_path)

            if result_image is not None:
                # Display results
                plt.figure(figsize=(10, 8))
                plt.imshow(result_image)
                plt.title(f"Webcam Photo {i+1} - Face Detection")
                plt.axis('off')
                plt.show()
            else:
                print(" No faces detected or error processing image")

            # Wait between photos (except after the last one)
            if i < 2:
                print(" Waiting 3 seconds for next photo...")
                time.sleep(3)

        except Exception as e:
            print(f" Error capturing photo {i+1}: {e}")
            continue

# Run webcam face detection
capture_and_detect_faces()

#Advanced Multi-Method Face Detection

In [13]:
def compare_face_detection_methods(image_path):
    """Compare different face detection methods"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            print("Error: Could not load image")
            return

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Different Haar Cascade classifiers
        classifiers = {
            'Default': 'haarcascade_frontalface_default.xml',
            'Alt': 'haarcascade_frontalface_alt.xml',
            'Alt2': 'haarcascade_frontalface_alt2.xml'
        }

        results = {}

        for name, classifier_file in classifiers.items():
            try:
                classifier_path = cv2.data.haarcascades + classifier_file
                face_cascade = cv2.CascadeClassifier(classifier_path)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                results[name] = (len(faces), img.copy())

                # Draw faces on the copy
                for (x, y, w, h) in faces:
                    cv2.rectangle(results[name][1], (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(results[name][1], f"Face", (x, y-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            except Exception as e:
                print(f"Error with {name} classifier: {e}")
                results[name] = (0, img.copy())

        # Display results
        fig, axes = plt.subplots(1, len(results), figsize=(15, 5))

        if len(results) == 1:
            axes = [axes]  # Make it iterable if only one subplot

        for idx, (name, (count, result_img)) in enumerate(results.items()):
            result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            axes[idx].imshow(result_rgb)
            axes[idx].set_title(f'{name}\n{count} faces detected')
            axes[idx].axis('off')

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error in comparison: {e}")

# Test different detection methods
print("Upload an image to compare face detection methods:")
compare_files = files.upload()
for filename in compare_files.keys():
    compare_face_detection_methods(filename)

#Real-time Face Detection System

In [13]:
class FaceDetectionSystem:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.detection_history = []

    def detect_faces(self, image_path, draw_landmarks=True):
        """Detect faces with optional landmark points"""
        img = cv2.imread(image_path)
        if img is None:
            return None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))

        # Record detection
        self.detection_history.append({
            'timestamp': time.time(),
            'faces_count': len(faces),
            'image_path': image_path
        })

        # Draw detailed annotations
        for (x, y, w, h) in faces:
            # Main bounding box
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Face center point
            center_x, center_y = x + w//2, y + h//2
            cv2.circle(img, (center_x, center_y), 3, (255, 0, 0), -1)

            # Face label with size
            face_size = w * h
            label = f"Face: {w}x{h}px"
            cv2.putText(img, label, (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # Draw landmark points (simplified)
            if draw_landmarks:
                landmarks = [
                    (x + w//4, y + h//3),    # Left eye
                    (x + 3*w//4, y + h//3),  # Right eye
                    (x + w//2, y + h//2),    # Nose
                    (x + w//3, y + 2*h//3),  # Left mouth
                    (x + 2*w//3, y + 2*h//3) # Right mouth
                ]

                for point in landmarks:
                    cv2.circle(img, point, 2, (0, 0, 255), -1)

        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def get_statistics(self):
        """Get detection statistics"""
        if not self.detection_history:
            return "No detections yet"

        total_faces = sum(entry['faces_count'] for entry in self.detection_history)
        avg_faces = total_faces / len(self.detection_history)

        return f"""
        Detection Statistics:
        - Total detections: {len(self.detection_history)}
        - Total faces found: {total_faces}
        - Average faces per image: {avg_faces:.1f}
        """

# Initialize the system
face_system = FaceDetectionSystem()

# Test the system
print("Testing advanced face detection system:")
print("Upload images for detailed face detection:")
test_files = files.upload()

for filename in test_files.keys():
    result = face_system.detect_faces(filename, draw_landmarks=True)
    if result is not None:
        plt.figure(figsize=(12, 8))
        plt.imshow(result)
        plt.title(f"Advanced Face Detection: {filename}")
        plt.axis('off')
        plt.show()

# Show statistics
print(face_system.get_statistics())

#Complete Real-time Demo

In [14]:
def complete_real_time_demo():
    """Complete real-time face detection demo"""
    print("COMPLETE REAL-TIME FACE DETECTION DEMO")
    print("=" * 50)

    # Initialize system
    system = FaceDetectionSystem()

    # Step 1: Webcam photos
    print("\n1. WEBCAM FACE DETECTION")
    print("You'll take 2 photos with your webcam")

    for i in range(2):
        print(f"\n   Preparing photo {i+1}/2...")
        filename = f'demo_photo_{i}.jpg'

        try:
            photo_path = take_photo(filename)
            print(" Photo captured!")

            # Detect faces
            result = system.detect_faces(photo_path, draw_landmarks=True)

            if result is not None:
                plt.figure(figsize=(10, 8))
                plt.imshow(result)
                plt.title(f"Real-time Detection {i+1}")
                plt.axis('off')
                plt.show()
            else:
                print("No faces detected")

            if i < 1:
                time.sleep(2)

        except Exception as e:
            print(f"Error: {e}")

    # Step 2: Upload additional test images
    print("\n2.UPLOAD ADDITIONAL TEST IMAGES")
    print("Upload more images for testing (optional)")

    uploaded = files.upload()
    for filename in uploaded.keys():
        result = system.detect_faces(filename, draw_landmarks=False)
        if result is not None:
            plt.figure(figsize=(10, 8))
            plt.imshow(result)
            plt.title(f"Uploaded Image: {filename}")
            plt.axis('off')
            plt.show()

    # Step 3: Show final statistics
    print("\n3. FINAL STATISTICS")
    print(system.get_statistics())

    print("\nDemo completed successfully!")

# Run the complete demo
complete_real_time_demo()